# fact_novelty

### Importación de librerías

In [ ]:
import yaml
import pandas as pd
from sqlalchemy import create_engine

### Conexión a base y bodega de datos

In [ ]:
with open('../config.yaml') as f:
    config = yaml.safe_load(f)
    configSource = config['source']
    configDestination = config['destination']

urlSource = f"{configSource['driver']}://{configSource['user']}:{configSource['password']}@{configSource['host']}:{configSource['port']}/{configSource['db']}"
urlDestination = f"{configDestination['driver']}://{configDestination['user']}:{configDestination['password']}@{configDestination['host']}:{configDestination['port']}/{configDestination['db']}"

engineSource = create_engine(urlSource)
engineDestination = create_engine(urlDestination)

### Extracción y transformación de datos

In [ ]:
novelties_services = pd.read_sql_table('mensajeria_novedadesservicio', engineSource)
dim_type_novelty = pd.read_sql_table('dim_type_novelty', engineDestination)
fact_novelty = novelties_services.groupby(['servicio_id', 'tipo_novedad_id']).size().reset_index(name='total_novelty')

fact_novelty.rename(columns={
    'servicio_id': 'key_service',
    'tipo_novedad_id': 'key_type_novelty',
}, inplace=True)

fact_novelty["key_fact_novelty"] = range(1, len(fact_novelty) + 1)

fact_novelty

In [ ]:
from datetime import date
fact_novelty['saved'] = date.today()

### Carga de datos

In [ ]:
fact_novelty.to_sql("fact_novelty", engineDestination, index=False, if_exists="replace")